# Preprocessing + Shared Artifacts (Vehicle Loan Default)



**Goal:** Create a *single, shared* processed dataset and a *shared split file* so both boosting models and neural networks train/evaluate on the same rows.



**Outputs written by this notebook:**

- `data/processed/loan_processed.csv` (features + `UNIQUEID` + `LOAN_DEFAULT`)

- `data/splits/split_uniqueid.csv` (maps `UNIQUEID` → `train`/`val`/`test`)



**Important:** This notebook does **not** do model-specific scaling/encoding. Those are fit **inside each modeling notebook using the training split only** to avoid leakage.


1️⃣ Import Required Libraries

In [8]:
# Data handling

import os

import re



import numpy as np

import pandas as pd



# Splitting

from sklearn.model_selection import StratifiedShuffleSplit



import matplotlib.pyplot as plt

import seaborn as sns



RANDOM_STATE = 42

DATA_PATH = os.path.join('..', 'data', 'train.csv')

SPLIT_PATH = os.path.join('..', 'data', 'splits', 'split_uniqueid.csv')

OUT_CSV = os.path.join('..', 'data', 'processed', 'loan_processed.csv')



os.makedirs(os.path.dirname(SPLIT_PATH), exist_ok=True)

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)



# Load labeled training data

df = pd.read_csv(DATA_PATH)

target_col = 'LOAN_DEFAULT'

id_col = 'UNIQUEID'



print('Shape:', df.shape)

display(df.head())


Shape: (233154, 41)


,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,DATE_OF_BIRTH,EMPLOYMENT_TYPE,...,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


**Why this notebook exists**



- We need a labeled holdout split (the provided `test.csv` is unlabeled).

- We save split membership by `UNIQUEID` so teammates can reproduce the exact same train/val/test rows.

- We keep preprocessing lightweight and portable by exporting a CSV that both boosting and NN can load.


## 1) Quick EDA: Target Variable


In [9]:
# --- EDA: target distribution (credit-risk imbalance is expected) ---

y = df[target_col].astype(int)

X = df.drop(columns=[target_col]).copy()



print('Target distribution (counts):')

display(y.value_counts(dropna=False))

print('Target distribution (rate):')

display(y.value_counts(normalize=True, dropna=False))


Target distribution (counts):


LOAN_DEFAULT
0    182543
1     50611
Name: count, dtype: int64

Target distribution (rate):


LOAN_DEFAULT
0    0.782929
1    0.217071
Name: proportion, dtype: float64

Why:



- Credit-risk datasets are typically imbalanced; checking the base default rate is essential.

- Confirms the target is clean (binary) before we define splits and artifacts.


## 2) Column Types (for later modeling)


In [10]:
# Identify categorical vs numeric columns (excluding the ID column)

feature_cols = [c for c in X.columns if c != id_col]



categorical_cols = [c for c in feature_cols if X[c].dtype == 'object']

numerical_cols = [c for c in feature_cols if c not in categorical_cols]



print('Total features (excluding ID):', len(feature_cols))

print('Numeric features:', len(numerical_cols))

print('Categorical features:', len(categorical_cols))

print('Example categoricals:', categorical_cols[:10])


Total features (excluding ID): 39
Numeric features: 33
Categorical features: 6
Example categoricals: ['DATE_OF_BIRTH', 'EMPLOYMENT_TYPE', 'DISBURSAL_DATE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH']


Why:



- Boosting models and neural networks often preprocess numeric/categorical features differently.

- We record which columns look categorical vs numeric so later notebooks can build consistent pipelines.


## 3) Shared Train / Validation / Test Split (Stratified)


In [11]:
# Reproducible split by UNIQUEID (shared with colleagues)

# Why:

# - We need a labeled holdout because provided test.csv is unlabeled.

# - We save membership by UNIQUEID so boosting and NN compare on identical rows.



def make_or_load_split_ids(df_raw: pd.DataFrame, path: str, seed: int = 42) -> pd.DataFrame:

    if os.path.exists(path):

        split_df = pd.read_csv(path)

        if set(split_df.columns) >= {id_col, 'split'}:

            return split_df[[id_col, 'split']]



    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=seed)

    idx_all = np.arange(len(df_raw))

    y_all = df_raw[target_col].astype(int).values

    (trainval_idx, test_idx), = sss1.split(idx_all, y_all)



    # Split the remaining 85% into 70% train and 15% val.

    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.1764705882, random_state=seed)

    (train_idx, val_idx), = sss2.split(trainval_idx, y_all[trainval_idx])

    train_idx = trainval_idx[train_idx]

    val_idx = trainval_idx[val_idx]



    split = pd.DataFrame({

        id_col: df_raw[id_col].values,

        'split': 'train',

    })

    split.loc[val_idx, 'split'] = 'val'

    split.loc[test_idx, 'split'] = 'test'



    split[[id_col, 'split']].to_csv(path, index=False)

    return split[[id_col, 'split']]



split_ids = make_or_load_split_ids(df, SPLIT_PATH, seed=RANDOM_STATE)

display(split_ids['split'].value_counts())



df_joined = df.merge(split_ids, on=id_col, how='left')

assert df_joined['split'].isna().sum() == 0



train_df = df_joined[df_joined['split'] == 'train'].copy()

val_df = df_joined[df_joined['split'] == 'val'].copy()

test_df = df_joined[df_joined['split'] == 'test'].copy()



print('Rows:', len(train_df), len(val_df), len(test_df))


split
train    163207
test      34974
val       34973
Name: count, dtype: int64

Rows: 163207 34973 34974


Why:



- Stratification preserves the default rate across splits.

- Validation is used for model selection/early stopping.

- Test is untouched until the final comparison.

- Saving the split by `UNIQUEID` makes results comparable across teammates/models.


## 4) Lightweight Feature Engineering (row-wise)


In [12]:
# Feature engineering (safe, row-wise)

# Why:

# - Adds domain-relevant signals (age, credit history length) without using future information.

# - Keeps output as a plain CSV usable by both boosting and NN notebooks.



_age_pat = re.compile(r'(?:(\d+)yrs)?\s*(?:(\d+)mon)?')



def age_str_to_months(s: str) -> float:

    if pd.isna(s):

        return np.nan

    s = str(s).strip()

    m = _age_pat.fullmatch(s)

    if not m:

        return np.nan

    yrs = int(m.group(1)) if m.group(1) else 0

    mon = int(m.group(2)) if m.group(2) else 0

    return yrs * 12 + mon



def add_engineered_features(frame: pd.DataFrame) -> pd.DataFrame:

    out = frame.copy()



    # Convert account-age style text fields into numeric months

    for col in ['AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH']:

        if col in out.columns:

            out[col + '_MONTHS'] = out[col].map(age_str_to_months)



    # Create age at disbursal (days) from two date columns

    if 'DATE_OF_BIRTH' in out.columns and 'DISBURSAL_DATE' in out.columns:

        dob = pd.to_datetime(out['DATE_OF_BIRTH'], format='%d-%m-%Y', errors='coerce')

        disb = pd.to_datetime(out['DISBURSAL_DATE'], format='%d-%m-%Y', errors='coerce')

        out['AGE_AT_DISBURSAL_DAYS'] = (disb - dob).dt.days



    # Drop raw date-like strings (models can use engineered numeric instead)

    for col in ['DATE_OF_BIRTH', 'DISBURSAL_DATE', 'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH']:

        if col in out.columns:

            out = out.drop(columns=[col])



    return out



df_processed = add_engineered_features(df)

print('Processed shape:', df_processed.shape)

display(df_processed.head())


Processed shape: (233154, 40)


,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,EMPLOYMENT_TYPE,STATE_ID,...,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,LOAN_DEFAULT,AVERAGE_ACCT_AGE_MONTHS,CREDIT_HISTORY_LENGTH_MONTHS,AGE_AT_DISBURSAL_DAYS
0,420825,50578,58400,89.55,67,22807,45,1441,Salaried,6,...,0,0,0,0,0,0,0,0,0,12633
1,537409,47145,65550,73.23,67,22807,45,1502,Self employed,6,...,0,1991,0,0,1,0,1,23,23,12110
2,417566,53278,61360,89.63,67,22807,45,1497,Self employed,6,...,0,0,0,0,0,0,0,0,0,12030
3,624493,57513,66113,88.48,67,22807,45,1501,Self employed,6,...,0,31,0,0,0,1,1,8,15,9066
4,539055,52378,60300,88.39,67,22807,45,1495,Self employed,6,...,0,0,0,0,0,1,1,0,0,14901


## 5) Re-check Column Types After Engineering


In [13]:
# Identify feature types on the processed dataframe

X_proc = df_processed.drop(columns=[target_col])



# Exclude ID from features

feature_cols = [c for c in X_proc.columns if c != id_col]

categorical_cols = [c for c in feature_cols if X_proc[c].dtype == 'object']

numerical_cols = [c for c in feature_cols if c not in categorical_cols]



print('Total features (excluding ID):', len(feature_cols))

print('Numeric:', len(numerical_cols))

print('Categorical:', len(categorical_cols))

print('Example categoricals:', categorical_cols[:10])


Total features (excluding ID): 38
Numeric: 36
Categorical: 2
Example categoricals: ['EMPLOYMENT_TYPE', 'PERFORM_CNS_SCORE_DESCRIPTION']


## 6) Export Shared Processed CSV


In [14]:
# Save processed dataset to CSV (shared artifact)

# Why:

# - Boosting models can consume this directly.

# - Neural nets can load it and do split + scaling/encoding inside the NN notebook.

# - Keeping this as CSV makes collaboration easier.



df_processed.to_csv(OUT_CSV, index=False)

print('Wrote:', OUT_CSV)


Wrote: ..\data\processed\loan_processed.csv


## 7) Split Sanity Checks


In [15]:
# Sanity checks: split file produces disjoint sets of UNIQUEID

split_ids = pd.read_csv(SPLIT_PATH)

train_ids = set(split_ids[split_ids['split'] == 'train'][id_col])

val_ids = set(split_ids[split_ids['split'] == 'val'][id_col])

test_ids = set(split_ids[split_ids['split'] == 'test'][id_col])



assert len(train_ids & val_ids) == 0

assert len(train_ids & test_ids) == 0

assert len(val_ids & test_ids) == 0



print('Split sizes:', len(train_ids), len(val_ids), len(test_ids))

print('No UNIQUEID overlap across splits')


Split sizes: 163207 34973 34974
No UNIQUEID overlap across splits


In [16]:
# How to use in modeling notebooks

print('Model notebooks should load:')

print(' -', OUT_CSV)

print(' -', SPLIT_PATH)


Model notebooks should load:
 - ..\data\processed\loan_processed.csv
 - ..\data\splits\split_uniqueid.csv


## 9) Final Sanity Check (Target)


In [17]:
# Final check: target distribution still imbalanced (expected)

dfp = df_processed[[id_col, target_col]].copy()

print('Processed rows:', len(dfp))

print('Default rate:', dfp[target_col].mean())

display(dfp[target_col].value_counts(normalize=True))


Processed rows: 233154
Default rate: 0.2170711203753742


LOAN_DEFAULT
0    0.782929
1    0.217071
Name: proportion, dtype: float64